In [61]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [91]:
def scrape_players_value(start, end):
    playersList, valuesList, clubsList, nationsList = [],[],[],[]
    
    user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'
    headers = {'User-Agent': user_agent}

    for k in range(int(start), int(end)):
        for i in range(0,10):
            pages = f'https://www.transfermarkt.co.uk/transfers/transferrekorde/statistik/top/saison_id/{k}/plus/1/galerie/0/page/{i+1}'
            pageTree = requests.get(pages, headers=headers)
            pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

            players = pageSoup.find_all("a", {"class": "spielprofil_tooltip"})
            values = pageSoup.find_all("td", {"class": "rechts hauptlink"})
            clubs = pageSoup.find_all('table', {'class': 'inline-table'})
            club_names, nation_names = [], []
            for i, club in enumerate(clubs):
                if (i+1)%3 == 0:
                    club_names.append(club.find_all('img', {'class':'tiny_wappen'})[0]['alt'])
                    nation_names.append(club.find_all('img', {'class':'flaggenrahmen'})[0]['alt'])

            for j in range(0,25):
                playersList.append(players[j].text)
                valuesList.append(values[j].text)
                clubsList.append(club_names[j])
                nationsList.append(nation_names[j])
                
    df = pd.DataFrame({"Players":playersList,
                       "Values":valuesList, 
                       'Clubs':clubsList, 
                       'Nations':nationsList})

    start_years = np.array([[f'{year}']*250 for year in range(int(start),int(end))]).flatten()
    end_years = np.array([[f'{year}']*250 for year in range(int(start)+1,int(end)+1)]).flatten()
    df['Start'] = start_years
    df['End'] = end_years
    
    import re
    def money(x):
        if re.search(pattern, x) is not None:
            return float(re.search(pattern, x)[0])
        else:
            return 0

    df['Values'] = df['Values'].apply(lambda x: money(x))
    
    return df

In [92]:
df = scrape_players_value('2005','2018')
df.to_csv("./values.csv")